## Import libraries

In [1]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import Flatten, Dense, Lambda
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import Adam
from keras.preprocessing import image

Using TensorFlow backend.


## Set variables

In [2]:
gen = image.ImageDataGenerator()
batch_size = 64 # was 64

## Import data

In [3]:
batches = gen.flow_from_directory('task3_data_mega/train',
                                  target_size=(299,299),
                                  class_mode='categorical',
                                  shuffle=True,
                                  batch_size=batch_size)

Found 64000 images belonging to 7 classes.


In [4]:
val_batches = gen.flow_from_directory('task3_data_mega/valid',
                                      target_size=(299,299),
                                      class_mode='categorical',
                                      shuffle=True,
                                      batch_size=batch_size)

Found 16000 images belonging to 7 classes.


## Import, configure, and compile model

In [5]:
# retrieve the full Keras Inception V3 model including imagenet weights
IV3 = InceptionV3(include_top=True, weights='imagenet',
                               input_tensor=None, input_shape=(299,299,3), pooling=None)

# define a new output layer to connect with the last fc layer
x = IV3.layers[-2].output
output_layer = Dense(7, activation='softmax', name='predictions')(x)

# combine the original model with the new output layer
IV3_2 = Model(inputs=IV3.input, outputs=output_layer)

# compile the new model
IV3_2.compile(optimizer=Adam(lr=0.0001), # was 0.0001
                loss='categorical_crossentropy', metrics=['accuracy'])

## Train 80K images

In [6]:
# 80K samples / 0.00008 LR
IV3_2.fit_generator(batches,
                   steps_per_epoch = batches.samples // batch_size,
                   validation_data = val_batches, 
                   validation_steps = val_batches.samples // batch_size,
                   epochs = 1)

Epoch 1/1
1125/1125 [==============================] - 685s 609ms/step - loss: 0.3675 - acc: 0.8669 - val_loss: 0.2256 - val_acc: 0.9134


In [7]:
# 80K samples / 0.00008 LR BEST MODEL SO FAR
IV3_2.fit_generator(batches,
                   steps_per_epoch = batches.samples // batch_size,
                   validation_data = val_batches, 
                   validation_steps = val_batches.samples // batch_size,
                   epochs = 1)

Epoch 1/1
1125/1125 [==============================] - 671s 597ms/step - loss: 0.0915 - acc: 0.9679 - val_loss: 0.0902 - val_acc: 0.9660


In [8]:
IV3_2.save('task3_models/iv3_80K.h5')

## Test 80K images

In [8]:
IV3_2 = load_model('task3_models/iv3_80K.h5')

In [9]:
test = gen.flow_from_directory('task3_data_mega/test_final_big',
                                target_size=(299,299),
                                class_mode='categorical',
                                shuffle=False,
                                batch_size=batch_size)

Found 12096 images belonging to 1 classes.


In [10]:
f_names = np.array(test.filenames)
predictions = np.array(IV3_2.predict_generator(test))
iv3_predictions = np.column_stack((f_names, predictions))
np.savetxt("iv3_predictions_final_big.csv", iv3_predictions, delimiter=",", fmt="%s")